In [1]:
from keras.models import load_model
from metrics import Pearson
from utils import prepare_data
import tensorflow_probability as tfp
from metrics import pearson, Pearson
import tensorflow as tf
import numpy as np
from dataloaders import GeneDataLoader
from utils import prepare_data, set_variables
from models import CNN
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Nadam
from keras.losses import KLDivergence, CategoricalCrossentropy
from plotting import plot_line_graph, multiplot_pearson
from utils import prepare_data, set_variables
from metrics import pearson, Pearson
from models import CNN
from plotting import box_plot
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
import numpy as np
#from keras.utils import to_categorical
from keras.utils import to_categorical
import itertools
from collections import OrderedDict
import os
import sys
#from keras.utils import pad_sequences
from keras.utils import pad_sequences
import tensorflow
from sklearn.model_selection import KFold, StratifiedKFold
from dataloaders.GeneWrapper import Gene_Wrapper
from sklearn.model_selection import train_test_split
#from plotting import plot_line_graph, box_plot, multiplot_pearson
from utils import read_model_file
import matplotlib.pyplot as plt


In [2]:
# Defining a few parameters
max_len = 34526
nb_filters = 32
filters_length = 10
pooling_size = 3
lstm_units = 32
lower_bound = 0
upper_bound = 4000
nb_classes = 9 # because we have 9 localisations
batch_size = 256

output_path = "/outputs" #"~/Downloads/model_outputs"
model_architecture_path = "model_architecture_viz/simple_cnn.png"

In [4]:
# Initializing test set

colab = False  #### Set colab flag ####

if colab:
    np.random.seed(3)
    url = 'https://www.dropbox.com/s/0r8nmwbthhkf2zi/seq_from_prim_and_icshape_withchrm_no_scaff.csv?dl=1' #'https://www.dropbox.com/s/hv4uau8q4wwg00k/final_data.csv?dl=1'
    data_org = pd.read_csv(url)
    test_data = data_org.sample(frac=0.1)
    train_data = test_data # data_org.drop(test_data.index)
else:
    np.random.seed(3)
    data_org = pd.read_csv('~/Downloads/seq_from_prim_and_icshape_withchrm_no_scaff.m6a.csv')
    test_data = data_org.sample(frac=0.1)
    train_data = test_data #  data_org.drop(test_data.index) # TODO: note: we also have to preprocess the test set similary
    # TODO: colab

data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13795,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13796,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13797,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13798,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


In [5]:

sum_vec = train_data.iloc[:, :9].sum(axis=1)
data2 = train_data.iloc[:, :9].divide(sum_vec, axis='index')
train_data_no_struct = pd.concat([data2, train_data['seq']], axis=1)
train_data_no_struct



,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
6355,0.250979,0.075937,0.090918,0.031291,0.101839,0.065864,0.066836,0.067107,0.249230,AGCCCTCCTCCCCTTTCCCCGCCCACTCCGGGCTGGCCTTATCCAG...
8099,0.166783,0.096917,0.093038,0.058871,0.135828,0.079169,0.092862,0.096614,0.179918,TTTTCTGTAAGGAGCATACTGTGCCCATTTATTATAGAATGCAGTT...
13459,0.148447,0.154639,0.091811,0.065952,0.120058,0.093185,0.120541,0.114084,0.091283,TAAGACAAGAAACACAAGAATGGAATCAACAGTTCTGGGCAAACCA...
6516,0.150002,0.117176,0.123043,0.038326,0.156221,0.091653,0.104970,0.115880,0.102728,ATTTCCCGGCGTGCTGCCATCCGTCGGGTCGCGGCAGCCATTTTGT...
12445,0.080430,0.182928,0.099621,0.073031,0.151337,0.069066,0.123032,0.123693,0.096861,AGCGGACCGACGCGACACGCCGTGCGCCTCCGCGGCTGCGCTACGA...
...,...,...,...,...,...,...,...,...,...,...
2942,0.113324,0.122352,0.068577,0.066046,0.206932,0.067581,0.070903,0.156626,0.127659,CCCAGGGTTTGGCTTTTTATTGACACAAACACACAAAGGCAGCTGT...
3902,0.151357,0.098038,0.155820,0.043068,0.140396,0.080892,0.135971,0.078269,0.116190,TGAGACGGTCCACAACAAAGAGATATCTCTCTGAGAGCTCCCAACT...
6770,0.036545,0.168322,0.105665,0.085846,0.082607,0.145518,0.134652,0.140166,0.100679,TCCTTGCCATGTTTCACATTTAATAAGATTCTTTCACTTTAATGCA...
3853,0.062000,0.118716,0.146389,0.063062,0.107018,0.169032,0.131032,0.086782,0.115969,TTTTGTGAACATTTACTAATCTTTTATTTGCTTTTCTTGGCACTCA...


In [6]:

# One hot encode the 'seq' attribute of the above table
mapping = {
    'A': 0,
    'C': 1,
    'G': 2,
    'T': 3
}

mapping_localisations = {
    'ERM':  0,
    'KDEL': 1,
    'LMA':  2,
    'MITO': 3,
    'NES':  4,
    'NIK':  5,
    'NLS':  6,
    'NUCP': 7,
    'OMM':  8
}

one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one = train_data_no_struct['seq'].apply(one_hot_encode_lam)

data_one

# Now just injecting this modified 'seq' back into the pandas frame
data_one_no_struct =  pd.concat([train_data_no_struct.iloc[:, :9], data_one], axis=1)

data_one_no_struct


# Additional ordinal encoding of the 'seq' attribute

gene_data = train_data['seq']

def label_dist(dist):
    # TODO: what is this
    assert (len(dist) == 4)
    return np.array(dist) / np.sum(dist)

encoding_seq = OrderedDict([
    ('UNK', [0, 0, 0, 0]),
    ('A', [1, 0, 0, 0]),
    ('C', [0, 1, 0, 0]),
    ('G', [0, 0, 1, 0]),
    ('T', [0, 0, 0, 1]),
    ('N', [0.25, 0.25, 0.25, 0.25]),  # A or C or G or T
])

encoding_keys = list(encoding_seq.keys())
seq_encoding_vectors = np.array(list(encoding_seq.values()))
encoding_vectors = seq_encoding_vectors


In [7]:

X = pad_sequences([[encoding_keys.index(c) for c in gene] for gene in gene_data],
                    maxlen=max_len,
                    dtype=np.int8, value=encoding_keys.index('UNK'))  # , truncating='post')

y = data_one_no_struct[mapping_localisations.keys()].values

# See notes to extend this for secondary structure

encoding_keys


['UNK', 'A', 'C', 'G', 'T', 'N']

In [8]:
# Import NN
from models.RNAtracker import RNATracker
from metrics import pearson, Pearson
# Set paths for model output
try:
    os.makedirs('')
except Exception as e:
    print(str(e))

model_output_folder = output_path #'~/Downloads/model_outputs'
epochs = 15

list_indizes = [i for i in range(X.shape[0])]
list_indizes

[Errno 2] No such file or directory: ''


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [10]:


path = "/Users/jonasflor/Downloads/RNATRACKER2023-07-14_50epochs.h5"  # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

train_indices, test_indices = train_test_split(list_indizes, random_state=42, test_size=0.2)

model = RNATracker(max_len, nb_classes, model_output_folder, kfold_index=0)
#model.build_model(nb_filters=kwargs['nb_filters'], filters_length=kwargs['filters_length'],
#                          pooling_size=kwargs['pooling_size'], lstm_units=kwargs['lstm_units'],
#                          embedding_vec=encoding_vectors)
model.pearson_attr = pearson
model.pearson_class_attr = Pearson(name="class_pearson", return_dict=True)
model.pearson_class_attr_no_dict = Pearson(name="class_nodict_pearson", return_dict=False)
model.build_model_advanced_masking(nb_filters=nb_filters,
                                                filters_length=filters_length,
                                                pooling_size=pooling_size,
                                                lstm_units=lstm_units,
                                                embedding_vec=encoding_vectors,
                                                load_weights=True, 
                                                w_par_dir=path)

Advanced Masking


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 34526)]              0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 34526, 4)             24        ['input_2[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 34517, 32)            1280      ['embedding_1[0][0]']         
                                                                                                  
 max_pooling1d_2 (MaxPoolin  (None, 11505, 32)            0         ['conv1d_2[0][0]']            
 g1D)                                                                                       

# LOADING model

In [ ]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()
model_name = "CNN_Baseline_4Conv_Struct_ext_struct_m6a" # "steve_CNN_Att10" #'CNN_architecture7'
model_architecture_path, model_output_path, params_dataLoader_train, params_dataLoader_valid, params_model, params_train = set_variables(model_name, max_seq_len)


In [22]:
#dataloader = GeneDataLoader(test_data, padding_length=train_data['seq'].apply(lambda x: len(x)).max(), shuffle=False, struct=True)
from itertools import islice
def batched(iterable, n):
   it = iter(iterable)
   while (batch := list(islice(it, n))):
       yield batch
prediction = []
y_test = []
# X , y list_indizes  
chunks = list(batched(list_indizes, 256))

for i in chunks: # i is a batch of indizes 
    prediction.append(model.model.predict(X[i]))
    y_test.append(y[i])

prediction = np.concatenate(prediction)
y_test = np.concatenate(y_test)
y_test = np.float32(y_test)

4/4 [==============================] - 1s 153ms/step


In [23]:
type(y_test[0, 0])

numpy.float32

In [24]:
def pearson_dict(y_true, y_pred):
    y_true_std = tfp.stats.stddev(y_true, sample_axis=0, keepdims=True)
    y_pred_std = tfp.stats.stddev(y_pred, sample_axis=0, keepdims=True)
    
    y_true /= (y_true_std + 1e-3)
    y_pred /= (y_pred_std + 1e-3)
    
    result = tfp.stats.covariance(x=y_true,
                                y=y_pred,
                                event_axis=None,
                                sample_axis=0,
                                keepdims=False)
    res_dict = {}
    res_dict['ERM'] = result[0]
    res_dict['KDEL'] = result[1]
    res_dict['LMA'] = result[2]
    res_dict['MITO'] = result[3]
    res_dict['NES'] = result[4]
    res_dict['NIK'] = result[5]
    res_dict['NLS'] = result[6]
    res_dict['NUCP'] = result[7]
    res_dict['OMM'] = result[8]
    return res_dict

In [25]:


pearson_dict(y_test, prediction)



{'ERM': <tf.Tensor: shape=(), dtype=float32, numpy=0.33111957>,
 'KDEL': <tf.Tensor: shape=(), dtype=float32, numpy=0.35920343>,
 'LMA': <tf.Tensor: shape=(), dtype=float32, numpy=0.11215091>,
 'MITO': <tf.Tensor: shape=(), dtype=float32, numpy=0.17782748>,
 'NES': <tf.Tensor: shape=(), dtype=float32, numpy=0.18748991>,
 'NIK': <tf.Tensor: shape=(), dtype=float32, numpy=0.21621695>,
 'NLS': <tf.Tensor: shape=(), dtype=float32, numpy=0.24022676>,
 'NUCP': <tf.Tensor: shape=(), dtype=float32, numpy=0.20533237>,
 'OMM': <tf.Tensor: shape=(), dtype=float32, numpy=0.16736066>}

In [26]:
np.mean(list(pearson_dict(y_test, prediction).values()))

0.2218809